In [64]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import random
import torch as th

In [36]:
train_images_path = Path("../data/tiny-imagenet-200/train/")

image_subpath = "images"

if not train_images_path.exists():
    raise FileNotFoundError("The train images folder does not exist.")

train_images_folder = [f for f in train_images_path.iterdir() if f.is_dir()]

len(train_images_folder)

200

In [ ]:
images_dict = {}
for folder in train_images_folder:
    images = list((folder / image_subpath).glob("*.JPEG"))
    selected_images = random.sample(images, 10)
    
    
    for image in selected_images:
        img = Image.open(image).convert("RGB")

        images_dict[image.stem] = transforms.ToTensor()(img)

dict_keys(['n02795169_97', 'n02795169_30', 'n02795169_358', 'n02795169_448', 'n02795169_437', 'n02795169_159', 'n02795169_245', 'n02795169_390', 'n02795169_244', 'n02795169_450', 'n02769748_243', 'n02769748_226', 'n02769748_422', 'n02769748_271', 'n02769748_487', 'n02769748_154', 'n02769748_437', 'n02769748_118', 'n02769748_290', 'n02769748_234', 'n07920052_409', 'n07920052_223', 'n07920052_334', 'n07920052_263', 'n07920052_146', 'n07920052_313', 'n07920052_371', 'n07920052_442', 'n07920052_322', 'n07920052_52', 'n02917067_85', 'n02917067_419', 'n02917067_158', 'n02917067_223', 'n02917067_381', 'n02917067_201', 'n02917067_96', 'n02917067_371', 'n02917067_496', 'n02917067_139', 'n01629819_8', 'n01629819_35', 'n01629819_450', 'n01629819_451', 'n01629819_123', 'n01629819_5', 'n01629819_131', 'n01629819_486', 'n01629819_150', 'n01629819_103', 'n02058221_30', 'n02058221_266', 'n02058221_411', 'n02058221_191', 'n02058221_375', 'n02058221_137', 'n02058221_14', 'n02058221_41', 'n02058221_338',

In [61]:
images_dict.items()

dict_items([('n02795169_97', tensor([[[0.3137, 0.4314, 0.4510,  ..., 0.3647, 0.1608, 0.3020],
         [0.3294, 0.4235, 0.4392,  ..., 0.3569, 0.2000, 0.3294],
         [0.3529, 0.4157, 0.4196,  ..., 0.3569, 0.2471, 0.3333],
         ...,
         [0.0471, 0.1725, 0.7098,  ..., 0.1647, 0.1373, 0.1216],
         [0.4902, 0.6353, 0.6039,  ..., 0.1294, 0.1373, 0.2157],
         [0.4980, 0.4667, 0.6118,  ..., 0.1176, 0.1569, 0.3255]],

        [[0.0902, 0.2078, 0.2275,  ..., 0.2314, 0.1412, 0.3333],
         [0.1059, 0.2078, 0.2157,  ..., 0.2353, 0.1804, 0.3647],
         [0.1373, 0.2118, 0.2039,  ..., 0.2353, 0.2314, 0.3804],
         ...,
         [0.0549, 0.1804, 0.7176,  ..., 0.1922, 0.1647, 0.1490],
         [0.4980, 0.6431, 0.6118,  ..., 0.1569, 0.1647, 0.2431],
         [0.5059, 0.4745, 0.6196,  ..., 0.1451, 0.1843, 0.3529]],

        [[0.1176, 0.2353, 0.2549,  ..., 0.3686, 0.3020, 0.5137],
         [0.1333, 0.2314, 0.2431,  ..., 0.3608, 0.3373, 0.5333],
         [0.1569, 0.2275, 0.2

In [62]:
class CustomImageDataset(Dataset):
    def __init__(self, data_dict):
        self.data = []
        self.names = []
        self.keys = list(data_dict.keys())
        
        for name, data in data_dict.items():
            self.data.append(data)
            self.classes = name.split("_")[0]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.classes[idx]
        
        return image, label
    
    
            

In [ ]:
dataset = CustomImageDataset(images_dict)

th.save(dataset, "../data/dataset_2000.pth")